# GraphReasoning with Autogen

Michael Yu-Chuan Hsu, MIT, 2024 mkychsu@MIT.EDU

Markus J. Buehler, MIT, 2024 mbuehler@MIT.EDU



In [1]:
!curl http://localhost:8080/v1/models

{"object":"list","data":[{"id":"/home/mkychsu/pool/llm/Meta-Llama-3.1-70B-Instruct-Q4_K_L.gguf","object":"model","owned_by":"me","permissions":[]}]}

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152

In [3]:
import autogen, openai
config_list = [
    {
        "model":"Llama3.1",
        "base_url": "http://localhost:8080/v1",
        "api_key":"NULL",
        "max_tokens": 40000
    },
]
config_list_semikong = config_list

# config_list_semikong = [
#     {
#         "model":"Semikong",
#         "base_url": "http://localhost:8081/v1",
#         "api_key":"NULL",
#         "max_tokens": 8192
#     },
# ]

# 
llm_config = {
    "cache_seed": 9527,  # seed for caching and reproducibility
    "config_list": config_list,  # a list of OpenAI API configurations
    "temperature": 0,  # temperature for sampling
    "max_tokens": 40000,
}

semikong_config = {
    "cache_seed": 9527,  # seed for caching and reproducibility
    "config_list": config_list_semikong,  # a list of OpenAI API configurations
    "temperature": 0,  # temperature for sampling
    "max_tokens": 40000,
}


/home/mkychsu/pool/.conda/envs/llm/lib/python3.12/site-packages/flaml/__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


In [4]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor

from tqdm.notebook import tqdm
from IPython.display import display, Markdown

verbatim=False

data_dir='./GRAPHDATA_TSMC_v1'    
data_dir_output='./GRAPHDATA_TSMC_OUTPUT_v1'

tokenizer_model=f'/home/mkychsu/pool/llm/SEMIKONG-8b-GPTQ'
# embedding_tokenizer = AutoTokenizer.from_pretrained(tokenizer_model, use_fast=False)
# embedding_model = AutoModelForCausalLM.from_pretrained(tokenizer_model, device_map='cuda', torch_dtype='auto', output_hidden_states=True)

embedding_tokenizer = AutoTokenizer.from_pretrained(tokenizer_model,use_fast=False, device_map="cuda:0")
embedding_model = AutoModelForCausalLM.from_pretrained(tokenizer_model,output_hidden_states=True).to('cuda:0')

embedding_doc_file=f'{data_dir_output}/chroma'


from PIL import Image

model_id = "/home/mkychsu/pool/llm/Cephalo-Phi-3-vision-128k-4b-alpha"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:1", trust_remote_code=True, torch_dtype="auto")
processor = AutoProcessor.from_pretrained(model_id, device_map="cuda:1", trust_remote_code=True) 




/home/mkychsu/pool/.conda/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mkychsu/pool/.conda/envs/llm/lib/python3.12/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/home/mkychsu/pool/.conda/envs/llm/lib/python3.12/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/home/mkychsu/pool/.conda/envs/llm/lib/python3.12/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torc

ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.


`low_cpu_mem_usage` was None, now default to True since model is quantized.
/home/mkychsu/pool/.conda/envs/llm/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:818: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(
INFO - Auto pick kernel based on compatibility: <class 'gptqmodel.nn_modules.qlinear.torch.TorchQuantLinear'>
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  2.43it/s]
INFO:transformers_modules.microsoft.Phi-3-vision-128k-instruct.c45209e90a4c4f7d16b2e9d48503c7f3e83623ed.image_embedding_phi3_v:learnable separator enabled for hd transform, hd_transform_order = sub_glb
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.45s/it]
/home/mkychsu/pool/.conda/envs/llm/lib/python3.12/site-

In [5]:
# from transformers import AutoProcessor 

# model_id = "/home/mkychsu/pool/llm/Cephalo-Phi-3-vision-128k-4b-alpha"

# model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0", trust_remote_code=True, torch_dtype="auto")
# processor = AutoProcessor.from_pretrained(model_id, device_map="cuda:0", trust_remote_code=True) 


In [6]:
from GraphReasoning import load_embeddings, save_embeddings, generate_node_embeddings
import networkx as nx

embedding_file='TSMC_KG_70b.pkl'
generate_new_embeddings=True

G = nx.read_graphml(f'{data_dir_output}/simple_graph_graphML_simplified.graphml')
relation = nx.get_edge_attributes(G, "title")
nx.set_edge_attributes(G, relation, "relation")
nx.set_node_attributes(G, nx.pagerank(G), "pr")

print(f'KG loaded: {G}')

import torch
if os.path.exists(f'{data_dir}/{embedding_file}'):
    generate_new_embeddings=False
    
with torch.no_grad():
    if generate_new_embeddings:

        # try:
        node_embeddings = generate_node_embeddings(G, embedding_tokenizer, embedding_model, )
        # except:
        #     node_embeddings = generate_node_embeddings(nx.DiGraph(), embedding_tokenizer, embedding_model, )

        save_embeddings(node_embeddings, f'{data_dir}/{embedding_file}')

    else:
        filename = f"{data_dir}/{embedding_file}"
        # file_path = hf_hub_download(repo_id=repository_id, filename=filename, local_dir='./')
        # print(f"File downloaded at: {file_path}")
        node_embeddings = load_embeddings(f'{data_dir}/{embedding_file}')

KG loaded: DiGraph with 4203 nodes and 4129 edges


In [7]:
import glob
import pandas as pd
from autogen.agentchat.contrib.vectordb.base import Document

chunks_list=sorted(glob.glob(f'{data_dir}/*chunks_clean.csv'))
chunk_ids = []
chunks=[]
titles=[]
for chunks_file in chunks_list:
    print(chunks_file)
    
    f = pd.read_csv(chunks_file)
    title = chunks_file.replace('_chunks_clean.csv', '').split('/')[-1]
    f['title'] = title
    chunk_ids += list(f['chunk_id'])
    chunks += list(f['text'])
    titles += list(f['title'])

docs = [
    Document(id=id, content=chunk, metadata={"title": title})
    for id, chunk, title in zip(chunk_ids, chunks, titles)
]

./GRAPHDATA_TSMC_v1/A_literature_review_on_variability_in_semiconductor_manufacturing_The_next_forward_leap_to_Industry_4.0_chunks_clean.csv
./GRAPHDATA_TSMC_v1/Atomic Layer Processing_semiconductor_chunks_clean.csv
./GRAPHDATA_TSMC_v1/Characterization of an Etch Profile at a Wafer Edge in Capacitively Coupled Plasma_chunks_clean.csv
./GRAPHDATA_TSMC_v1/Dependences of bottom and sidewall etch rates on biasvoltage and source power during the etching of poly-Si andfluorocarbon polymer usingSF6, C4F8,andO2plasmas_chunks_clean.csv
./GRAPHDATA_TSMC_v1/Etch Mechanism Study in Gate Patterning for 14 nm Node and_chunks_clean.csv
./GRAPHDATA_TSMC_v1/Future_of_plasma_etching_for_microelectronics_Challenges_chunks_clean.csv
./GRAPHDATA_TSMC_v1/High-density vertical sidewall MoS2 transistors through T-shape vertical lamination_chunks_clean.csv
./GRAPHDATA_TSMC_v1/Influence_of_sidewall_thickness_variation_on_transfer_characteristics_of_L-shaped_Impact-ionization_MOS_transistor_chunks_clean.csv
./GR

In [8]:
def custom_token_count_function(text, placeholder=''):
    return len(embedding_tokenizer.encode(text))

In [9]:
import importlib
from typing import Optional, Union, cast, TypeVar

import numpy as np
import numpy.typing as npt

from chromadb.api.types import EmbeddingFunction, Embeddings
from chromadb import PersistentClient
import torch

import networkx as nx

Embeddable = Union[str, nx.DiGraph]
D = TypeVar("D", bound=Embeddable, contravariant=True)

class TransformerEmbeddingFunction(EmbeddingFunction[D]):
    def __init__(
            self,
            embedding_tokenizer,
            embedding_model,
            cache_dir: Optional[str] = None,
    ):
        try:
            from transformers import AutoModel, AutoTokenizer

            self._torch = importlib.import_module("torch")
            self._tokenizer = embedding_tokenizer #AutoTokenizer.from_pretrained(model_name)
            self._model = embedding_model #AutoModelForCausalLM.from_pretrained(model_name, device_map="cuda:0", cache_dir=cache_dir)
        except ImportError:
            raise ValueError(
                "The transformers and/or pytorch python package is not installed. Please install it with "
                "`pip install transformers` or `pip install torch`"
            )

    @staticmethod
    def _normalize(vector: npt.NDArray) -> npt.NDArray:
        """Normalizes a vector to unit length using L2 norm."""
        norm = np.linalg.norm(vector)
        if norm == 0:
            return vector
        return vector / norm

    def __call__(self, input: D) -> Embeddings:
        inputs = self._tokenizer(
            input, padding=True, truncation=True, return_tensors="pt"
        ).to('cuda')
        with self._torch.no_grad():
            outputs = self._model(**inputs)
        try:
            embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()  # mean pooling
        except:
            embeddings = outputs.hidden_states[-1].mean(dim=1).detach().to(torch.float).cpu().numpy()
            
        return [e.tolist() for e in self._normalize(embeddings)]
            
embedding_function = TransformerEmbeddingFunction(embedding_tokenizer=embedding_tokenizer, embedding_model=embedding_model)
client = PersistentClient(path=embedding_doc_file)


INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [10]:
os.environ["CHROMADB_MAX_BATCH_SIZE"]="10" # only support single GPU, "20" is for V100 32G. For A100 80G, try "50'
from autogen.agentchat.contrib.vectordb.chromadb import ChromaVectorDB

ChromaDB=ChromaVectorDB(client=client, embedding_function=embedding_function)

try:
    client.get_collection('TSMC_70b', embedding_function=embedding_function)
    ChromaDB.active_collection=client.get_or_create_collection('TSMC_70b', embedding_function=embedding_function)
except:
    client.get_or_create_collection('TSMC_70b', embedding_function=embedding_function)
    ChromaDB.active_collection=client.get_or_create_collection('TSMC_70b', embedding_function=embedding_function)
    ChromaDB.insert_docs(docs=docs, collection_name='TSMC_70b', upsert=True)
# try:
#     ChromaDB.active_collection=client.get_or_create_collection('5books_70b', embedding_function=embedding_function)
#     print(ChromaDB.retrieve_docs('Hello World!'))
# except:
#     pass    
#     client.delete_collection('5books_70b')
#     ChromaDB.insert_docs(docs=docs, collection_name='5books_70b', upsert=False)

In [20]:
system_prompt = '''
You are a scientific assistant extracting information from figures.
Return all the features you observe on the image.
'''
from pathlib import Path

def generate_figure(image, system_prompt=system_prompt, 
                prompt="", model=model, processor=processor, temperature=0,
                           ):
    if system_prompt==None:
        messages=[
            {"role": "user", "content": f"Here is the image: <|image_1|>.\n" + prompt},
        ]

    else:
        messages=[
            {"role": "system",  "content": system_prompt},
            {"role": "user", "content": f"Here is the image: <|image_1|>.\n" + prompt},
        ]
    try:
        print(f'processing image: {image}')
        
        pwd = os.getcwd()
        image = image.split(pwd)[-1]
        image=Path('.').glob(f'**/{image}', case_sensitive=False)
        image = list(image)[0]
        print(f'successfully processed image: {image}')
        
    except:
        print(f'fail to processing image: {image}')
        return '' 
    image = Image.open(image)
    print(f'Extracting infomation from {image}')
    prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(prompt, [image], return_tensors="pt").to("cuda:1") 
    generation_args = { 
                        "max_new_tokens": 1024, 
                        "temperature": 0.1, 
                        "do_sample": True, 
                        "stop_strings": ['<|end|>',
                                         '<|endoftext|>'],
                        "tokenizer": processor.tokenizer,
                      } 

    generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args) 

    # remove input tokens 
    generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
    return processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 
    

In [21]:
import hashlib
import os
import re
import uuid
from typing import Any, Callable, Dict, List, Literal, Optional, Tuple, Union
from IPython import get_ipython
from GraphReasoning import collect_entities

'''
try:
    import chromadb
except ImportError as e:
    raise ImportError(
        f"{e}. You can try `pip install autogen-agentchat[retrievechat]~=0.2`, or install `chromadb` manually."
    )
''' 
from autogen.agentchat import UserProxyAgent
from autogen.agentchat.agent import Agent
from autogen.agentchat.contrib.vectordb.base import Document, QueryResults, VectorDB, VectorDBFactory
from autogen.agentchat.contrib.vectordb.utils import (
    chroma_results_to_query_results,
    filter_results_by_distance,
    get_logger,
)
from autogen.code_utils import extract_code
from autogen.retrieve_utils import (
    TEXT_FORMATS,
    query_vector_db,
)
from autogen.token_count_utils import count_token

from autogen.formatting_utils import colored

logger = get_logger(__name__)


class HybridGraphRAGAgent(UserProxyAgent):
    """(In preview) The Graph Retrieval-Augmented User Proxy retrieves information from knowledge graphs based on the embedding
    similarity, and sends them along with the question to this or next assistant
    """

    def __init__(
        self,
        name="HybridGraphRAGAgent",  # default set to RetrieveChatAgent
        human_input_mode: Literal["ALWAYS", "NEVER", "TERMINATE"] = "NEVER",
        is_termination_msg: Optional[Callable[[Dict], bool]] = None,
        retrieve_config: Optional[Dict] = None,  # config for the retrieve agent
        **kwargs,
    ):
        r"""
        Args:
            name (str): name of the agent.

            human_input_mode (str): whether to ask for human inputs every time a message is received.
                Possible values are "ALWAYS", "TERMINATE", "NEVER".
                1. When "ALWAYS", the agent prompts for human input every time a message is received.
                    Under this mode, the conversation stops when the human input is "exit",
                    or when is_termination_msg is True and there is no human input.
                2. When "TERMINATE", the agent only prompts for human input only when a termination
                    message is received or the number of auto reply reaches
                    the max_consecutive_auto_reply.
                3. When "NEVER", the agent will never prompt for human input. Under this mode, the
                    conversation stops when the number of auto reply reaches the
                    max_consecutive_auto_reply or when is_termination_msg is True.

            is_termination_msg (function): a function that takes a message in the form of a dictionary
                and returns a boolean value indicating if this received message is a termination message.
                The dict can contain the following keys: "content", "role", "name", "function_call".

            retrieve_config (dict or None): config for the retrieve agent.

                To use default config, set to None. Otherwise, set to a dictionary with the
                following keys:
                - `vector_db` (VectorDB) - the exisiting vector db for the retrieve chat.
                    It should be an instance of the VectorDB protocol.
                - `context_max_tokens` (Optional, int) - the context max token size for the
                    retrieve chat.
                    If key not provided, a default size `max_tokens * 0.8` will be used.
                - `custom_token_count_function` (Optional, Callable) - a custom function to count the
                    number of tokens in a string.
                    The function should take (text:str, model:str) as input and return the
                    token_count(int). the retrieve_config["model"] will be passed in the function.
                    Default is autogen.token_count_utils.count_token that uses tiktoken, which may
                    not be accurate for non-OpenAI models.
                - `distance_threshold` (Optional, float) - the threshold for the distance score, only
                    distance smaller than it will be returned. Will be ignored if < 0. Default is -1.
                - `_knowledge_graph` (Optional, nx.DiGraph) - a knowledge graph for RAG

            `**kwargs` (dict): other kwargs in [UserProxyAgent](../user_proxy_agent#__init__).
        """
        super().__init__(
            name=name,
            human_input_mode=human_input_mode,
            **kwargs,
        )
        self._retrieve_config = {} if retrieve_config is None else retrieve_config
        self._task = self._retrieve_config.get("task", "default")
        self._vector_db = self._retrieve_config.get("vector_db", None)
        self._model = self._retrieve_config.get("model", None)
        self._max_tokens = self._retrieve_config.get("max_tokens", 8000)
        self._knowledge_graph = self._retrieve_config.get("knowledge_graph", None)
        self.customized_prompt = self._retrieve_config.get("customized_prompt", None)
        self.custom_token_count_function = self._retrieve_config.get("custom_token_count_function", count_token)
        self._context_max_tokens = self._retrieve_config.get("context_max_tokens", self._max_tokens * 0.8)
        self._n_results = self._retrieve_config.get("n_results", 5)
        self._distance_threshold = self._retrieve_config.get("distance_threshold", -1)

        self.generate_figure = self._retrieve_config.get("generate_figure", None)
        self._ipython = get_ipython()
        self._results = []  # the results of the current query
        self._intermediate_answers = set()  # the intermediate answers
        self._doc_contents = []  # the contents of the current used chunk
        self._doc_ids = []  # the ids of the current used chunk 
        self._current_docs_in_context = []  # the ids of the current context sources
        self.register_reply(Agent, HybridGraphRAGAgent._generate_retrieve_user_reply, position=2)

        if not isinstance(self._vector_db, VectorDB):
            logger.error('You must provide an instance of vectordb')
            return

    def _reset(self, intermediate=False):
        # self._doc_idx = -1  # the index of the current used doc
        self._results = []  # the results of the current query
        if not intermediate:
            self._intermediate_answers = set()  # the intermediate answers
            self._doc_contents = []  # the contents of the current used doc
            self._doc_ids = []  # the ids of the current used doc
            
#     def graphRAG(self, id):
#         edges = list(self._knowledge_graph.out_edges(data=True))
#         nodes = set()
        
#         # for info in self._results[0]:
#         #     id = info[0]['id']
# #             if 'community_summary' in id:
# #                 nodes |= set(eval(doc['metadata']['community']))
                
# #             else:
#         for edge in edges:
#             if edge[2]['chunk_id'] == id:
#                 nodes.add(edge[0])
#                 nodes.add(edge[1])
#         return collect_entities(self._knowledge_graph.subgraph(nodes))
    
    def graphRAG(self, id):  
        edges = list(self._knowledge_graph.out_edges(data=True))
        nodes = set()
        
        # for info in self._results[0]:
        #     id = info[0]['id']
#             if 'community_summary' in id:
#                 nodes |= set(eval(doc['metadata']['community']))
#             else:
        for edge in edges:
            if edge[2]['chunk_id'] == id:
                nodes.add(edge[0])
                nodes.add(edge[1])
                
        figure_info=''
        for node in nodes:
            if 'png' in node:
                figure_info += generate_figure(node)
        return collect_entities(self._knowledge_graph.subgraph(nodes)) + f' Here is information from the figure: {figure_info}'
    
    def _get_context(self):
        doc_contents = ""
        self._current_docs_in_context = []
        current_tokens = 0
        # _doc_idx = self._doc_idx
        for idx, doc in enumerate(self._results[0]):
            doc = doc[0]
            # if idx <= _doc_idx:
            #     continue
            # if doc["id"] in self._doc_ids:
            #     continue
            graph_results = self.graphRAG(doc['id'])
            _doc_tokens = self.custom_token_count_function(doc["content"]+graph_results) 
            
#             # if _doc_tokens > self._context_max_tokens:
#             #     func_print = f"Skip doc_id {doc['id']} as it is too long to fit in the context."
#             #     print(colored(func_print, "green"), flush=True)
#             #     # self._doc_idx = idx
#             #     continue
            
#             if current_tokens + _doc_tokens > self._context_max_tokens - 100:
#                 break

            func_print = f"Adding content of doc {doc['id']} to context from: {doc['metadata']['title']}"
            print(colored(func_print, "green"), flush=True)

            # doc_contents += doc["content"] + "\n"
            # _metadata = doc.get("metadata")
            # if isinstance(_metadata, dict):
                # self._current_docs_in_context.append(_metadata.get("source", ""))
            
            # self._doc_idx = idx
            # self._doc_ids.append(doc["id"])
            # self._doc_contents.append(doc["content"])
            doc_contents += f"The following information related to your question is from TITLE: {' '.join(doc['metadata']['title'].split('_'))} "
            doc_contents += f"Source text: {doc['content']}\n"     
            doc_contents += f"Relationships of the knowledge: {self.graphRAG(doc['id'])}\n"
            
            current_tokens += _doc_tokens
            func_print = f"Current tokens in use: {current_tokens}"
            print(colored(func_print, "green"), flush=True)

        return doc_contents

    def _generate_retrieve_user_reply(
        self,
        messages: Optional[List[Dict]] = None,
        sender: Optional[Agent] = None,
        config: Optional[Any] = None,
    ) -> Tuple[bool, Union[str, Dict, None]]:
        """In this function, we will update the context and reset the conversation based on different conditions.
        We'll update the context and reset the conversation if update_context is True and either of the following:
        """
        # if config is None:
        #     config = self
        # if messages is None:
        #     messages = self._oai_messages[sender]
        # _message = messages[-1]
        
        
        self._reset(intermediate=True)

        response = self.generate_oai_reply(messages=[self._oai_messages[sender][-1]], sender=sender)
        
        problems = response[1].split('\n')
        
        final_response = ''
        
#         for problem in problems:
#             if len(problem) <= 5:
#                 continue
#             if "?" not in problem:
#                 continue
#             final_response += f"Let's first solve sub-question: {problem}\n"
#             self.retrieve_docs(
#                 problem=problem,
#             )

#             final_response += self._get_context()
            
#             self.clear_history(sender)
#             sender.clear_history(self)        
#             break

        problem = problems[-1]
        final_response += f"{problem}\n"
        self.retrieve_docs(
            problem=problem,
        )

        final_response += self._get_context()

        self.clear_history(sender)
        sender.clear_history(self)        


        return True, final_response # self._generate_message(doc_contents, problem=_message, task=self._task)


    def retrieve_docs(self, problem: Union[str, List[str]] = None):
        """Retrieve docs based on the given problem and assign the results to the class property `_results`.
        The retrieved docs should be type of `QueryResults` which is a list of tuples containing the document and
        the distance.

        Args:
            problem (str): the problem to be solved.
            n_results (int): the number of results to be retrieved. Default is 20.

        Returns:
            None.
        """
        print(colored("Retrieving for:", "green"), end = " ")
        print(f"{problem}")
        
        if isinstance(self._vector_db, VectorDB):
            kwargs = {}
            results = self._vector_db.retrieve_docs(
                queries=[problem],
                n_results=self._n_results,
                distance_threshold=self._distance_threshold,
                **kwargs,
            )
            self._results = results
            
            # print("VectorDB returns doc_ids: ", [[r[0]["id"] for r in rr] for rr in results])
            return

        

In [22]:
from openai import OpenAI
from GraphReasoning import extract_keywords_to_nodes, find_shortest_path_subgraph_between_nodes
# local_search(question, generate, graph, node_embeddings, embedding_tokenizer, embedding_model, N_samples=5, similarity_threshold=0.9)
class llm:
    def __init__(self, llm_config):
        self.client = OpenAI(api_key=llm_config["api_key"],
                             base_url=llm_config["base_url"],
                             )
        self.model = llm_config["model"]
        self.max_tokens = llm_config["max_tokens"]
        
    def generate_cli(self, system_prompt="You are an expert in this field. Try your best to give a clear and concise answer.", 
                           prompt="Hello world! I am", temperature=0,
                           ):     
        try:
            if system_prompt==None:
                messages=[
                    {"role": "user", "content": prompt},

                ]

            else:
                messages=[
                    {"role": "system",  "content": system_prompt},
                    {"role": "user", "content": prompt},

                ]
            result=self.client.chat.completions.create(
                    model=self.model,
                    messages=messages,
                    temperature=temperature,
                    max_tokens=self.max_tokens,
                )

            return result.choices[0].message.content
        except:
            return ''
        
llm=llm(llm_config=config_list[0])        
generate = llm.generate_cli

class GraphRAGAgent(UserProxyAgent):
    """(In preview) The Graph Retrieval-Augmented User Proxy retrieves information from knowledge graphs based on the embedding
    similarity, and sends them along with the question to this or next assistant
    """

    def __init__(
        self,
        name="GraphRAGChatAgent",  # default set to RetrieveChatAgent
        human_input_mode: Literal["ALWAYS", "NEVER", "TERMINATE"] = "NEVER",
        is_termination_msg: Optional[Callable[[Dict], bool]] = None,
        generate=None,
        node_embeddings=None,
        embedding_tokenizer=None,
        embedding_model=None,
        retrieve_config: Optional[Dict] = None,  # config for the retrieve agent
        **kwargs,
    ):
        r"""
        Args:
            name (str): name of the agent.

            human_input_mode (str): whether to ask for human inputs every time a message is received.
                Possible values are "ALWAYS", "TERMINATE", "NEVER".
                1. When "ALWAYS", the agent prompts for human input every time a message is received.
                    Under this mode, the conversation stops when the human input is "exit",
                    or when is_termination_msg is True and there is no human input.
                2. When "TERMINATE", the agent only prompts for human input only when a termination
                    message is received or the number of auto reply reaches
                    the max_consecutive_auto_reply.
                3. When "NEVER", the agent will never prompt for human input. Under this mode, the
                    conversation stops when the number of auto reply reaches the
                    max_consecutive_auto_reply or when is_termination_msg is True.

            is_termination_msg (function): a function that takes a message in the form of a dictionary
                and returns a boolean value indicating if this received message is a termination message.
                The dict can contain the following keys: "content", "role", "name", "function_call".

            retrieve_config (dict or None): config for the retrieve agent.

                To use default config, set to None. Otherwise, set to a dictionary with the
                following keys:
                - `vector_db` (VectorDB) - the exisiting vector db for the retrieve chat.
                    It should be an instance of the VectorDB protocol.
                - `context_max_tokens` (Optional, int) - the context max token size for the
                    retrieve chat.
                    If key not provided, a default size `max_tokens * 0.8` will be used.
                - `custom_token_count_function` (Optional, Callable) - a custom function to count the
                    number of tokens in a string.
                    The function should take (text:str, model:str) as input and return the
                    token_count(int). the retrieve_config["model"] will be passed in the function.
                    Default is autogen.token_count_utils.count_token that uses tiktoken, which may
                    not be accurate for non-OpenAI models.
                - `distance_threshold` (Optional, float) - the threshold for the distance score, only
                    distance smaller than it will be returned. Will be ignored if < 0. Default is -1.
                - `_knowledge_graph` (Optional, nx.DiGraph) - a knowledge graph for RAG

            `**kwargs` (dict): other kwargs in [UserProxyAgent](../user_proxy_agent#__init__).
        """
        super().__init__(
            name=name,
            human_input_mode=human_input_mode,
            **kwargs,
        )
        self._retrieve_config = {} if retrieve_config is None else retrieve_config
        self._knowledge_graph = self._retrieve_config.get("knowledge_graph", None)
        self._n_results = self._retrieve_config.get("n_results", 5)
        self._distance_threshold = self._retrieve_config.get("distance_threshold", 0.9)
        
        self.generate = generate
        self.node_embeddings = node_embeddings
        self.embedding_tokenizer = embedding_tokenizer
        self.embedding_model = embedding_model
        
        self._ipython = get_ipython()
        self._results = []  # the results of the current query
        self._intermediate_answers = set()  # the intermediate answers
        self.register_reply(Agent, GraphRAGAgent._generate_retrieve_user_reply, position=2)

    def _reset(self, intermediate=False):
        # self._doc_idx = -1  # the index of the current used doc
        self._results = []  # the results of the current query
        if not intermediate:
            self._intermediate_answers = set()  # the intermediate answers
            self._doc_contents = []  # the contents of the current used doc
            self._doc_ids = []  # the ids of the current used doc
            
    def graphRAG(self, message):  
        nodes = extract_keywords_to_nodes(message, self.generate, self.node_embeddings, self.embedding_tokenizer, self.embedding_model, self._n_results, similarity_threshold=self._distance_threshold)
        subgraph = find_shortest_path_subgraph_between_nodes(self._knowledge_graph.to_undirected(), nodes)
        return collect_entities(self._knowledge_graph.subgraph(subgraph))


    def _generate_retrieve_user_reply(
        self,
        messages: Optional[List[Dict]] = None,
        sender: Optional[Agent] = None,
        config: Optional[Any] = None,
    ) -> Tuple[bool, Union[str, Dict, None]]:
        """In this function, we will update the context and reset the conversation based on different conditions.
        We'll update the context and reset the conversation if update_context is True and either of the following:
        """
        # if config is None:
        #     config = self
        # if messages is None:
        #     messages = self._oai_messages[sender]
        # _message = messages[-1]
        
        
        self._reset(intermediate=True)

        
        relationships = self.graphRAG(self._oai_messages[sender][-1])
                
        final_response = f"Please consider the following relationships of the knowledge related to the question and make your response: {relationships}"

        self.clear_history(sender)
        sender.clear_history(self)        

        return True, final_response # self._generate_message(doc_contents, problem=_message, task=self._task)


In [23]:
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the engineer to discuss the QA result.",
    human_input_mode="NEVER",
    code_execution_config=False,
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", "").replace("*", "").rstrip(), 
    # is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
)

graph_rag_agent = GraphRAGAgent(
    name="graph_rag_agent",
    system_message="""RAG agent. 
""",
    human_input_mode="NEVER",
    code_execution_config=False,
    generate = generate,
    node_embeddings = node_embeddings,
    embedding_tokenizer = embedding_tokenizer,
    embedding_model = embedding_model,
    retrieve_config={
        "n_results": 5,
        "knowledge_graph": G,
        "distance_threshold": 1.5
    },
    llm_config=llm_config,
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", "").replace("*", "").rstrip(),
)

planner = autogen.AssistantAgent(
    name="Planner",
    llm_config=llm_config,
    system_message="""Planner, more of a supervising project manager who works with other engineers but only has basic understanding about semiconductors. Don't write code.
You will come up a concise but through plan, breaking down the problem into several sub-questions that can help solve it step by step. These will be the core of the discussion.
Don't make assumption. Only adopt the ideas from other agents. Don't write citation or references
For example, when you try to plan an etching task, your answer should consider some break-down subquestions in mind like the example below: 
1. What kind of gas we are using is one of the most important issue which can lead to different side effects
2. What selectivity ratio we are adopting is also critical as it can affect how aggresive the etching process is going.
3. Any suggeted side wall protection to improve the overall performance?
4. Any suggeted step to help debris reduction?
At the end of your response, always put the highest-priorty sub-question for the agents to answer in this format: 'QUESTION: ... ?'. 
Don't revise the sub-question when the critic gives a good score higher than 6/10 because that would waste our time. Just go ahead for the next sub-question.
When all the subquestions have been answered, add "WRITE REPORT" at the end of your response.
""",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", "").replace("*", "").rstrip(), 
)

# planner = autogen.AssistantAgent(
#     name="Planner",
#     llm_config=llm_config,
#     system_message="""Planner, more of a project manager who works with other engineers but only has basic understanding about semiconductors. Don't write code.
#     You will come up a concise plan, breaking down the problem into several sub-questions that can help solve it step by step. These will be the core of the discussion.
#     At the end of your response, always put the highest-priorty sub-question for the agents to answer in this format: 'QUESTION: ... ?'. 
#     If an answer scores higher than and equal to 7/10, proceed with the next sub-question.
#     Otherwise, modify that question and follow up to improve the answer only once, and then move on.
#     If you think all the sub-questions have been handled, which means the main problem can be solved with all the sub-answers combined, you should write a concise report that includes all the sub-questions and all the sub-answers while keeping the citations in academic style with the citations as is.
# """,
#     is_termination_msg=lambda x: "TERMINATE" in x.get("content", "").replace("*", "").rstrip(), 
# )


hgraph_rag_agent = HybridGraphRAGAgent(
    name="hybrid_graph_rag_agent",
    system_message="""RAG agent. 
First look at the message you recieve and extract the question of the highest priority, often starting with QUESTION: ... , and pass it on in question-only format, such as: QUESTION: ...?
There should be always a question to consider so do not make up questions. If all the questions seem to be solved already, just answer 'TERMINATE' only.
""",
    human_input_mode="NEVER",
    code_execution_config=False,
    # max_consecutive_auto_reply=5,
    # llm_config=llm_config,
    llm_config=llm_config,
    retrieve_config={
        "custom_token_count_function": custom_token_count_function,
        "vector_db": ChromaDB,
        "n_results": 5,
        "max_tokens": llm_config["max_tokens"],
        "knowledge_graph": G,
        "distance_threshold": 1.5
    },
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", "").replace("*", "").rstrip(), 
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message="""Engineer with semiconductor backgrounds. Don't write code.
Start your response with: QUESTION: ...? \n ANSWER: ... .
You should always use the information you recieve from the other agents and don't make assumption. You should keep references when you use the provided information from another agent
Write your answer strictly in academic style with citations such as '<something true> [1]' and a references section with [1] <REFERENCE TITLE>: <reasons> and following the number in all your answers to make sure your citation is not overlapping.
Don't ever cite any sources that are not from the information you have. If you have an idea that is hypothetical, only mark it in your response.
Don't indirect cite the reference from the source texts.
Add "\nCLEAR HISTORY graph_rag_agent" at the end of your reply
""",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", "").replace("*", "").rstrip(), 
)

critic = autogen.AssistantAgent(
    name="Critic",
    # llm_config=llm_config,
    llm_config=semikong_config,
    system_message="""Senior engineer critic with semiconductor background. Don't write code.
Concisely criticize or approve whether the current sub-answer from an agent can solve the question.
Fairly evalute the completeness of the answer to the question into score on a scale of 1 to 10, 6 being acceptable. Below is the score policy you should consider.
1. Credibility: 0 to 5/5 for how much content of the answer is aquired by the information from the knowledge graphs or source texts from agents?
2. Correctness: 0 to 5/5 for how good is the answer to address the question. Does it reason the answer well?
3. Creativity: this does not give points but please report whether the answer provides any new insights based on the reasoning results.
""",
    # no more scoring, just raise some potential issues for the summarizer to organize a report
    # planner do not revise or consider the suggestion from critic to ensure the chat flow.
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", "").replace("*", "").rstrip(), 
)

summarizer = autogen.AssistantAgent(
    name="Summarizer",
    llm_config=llm_config,
    system_message="""Engineer with semiconductor backgrounds. Don't write code.
You write a academic-style report about the task and its subquestions and subanswers discussed by all the agents. 
The format of your report should have all the components listed below with reorganized citations that you see only in the previous responses from the engineer agents:
1. The definition of the task 
2. Subquestions that can help solve the task and the corresponding subanswers numbering starting from a, b, c ... 
For example,
Sub-question a: ... . Sub-answer a: ...
Sub-question b: ... . Sub-answer b: ...
Sub-question c: ... . Sub-answer c: ...
3. Potential issues reported
4. Summary
5. References
You only summarize the information in the chat history. Don't add new questions or assumptions. You should keep references when you use the provided information from another agent
While writing the report strictly in academic style with citations such as '<something true> [1]' and a references section with [1] <REFERENCE TITLE>: <reasons> and following the number in all your answers to make sure your citation is not overlapping.
Don't ever cite any sources that are not from the information you have. If you have an idea that is hypothetical, only mark it in your response.
If you think all the sub-questions have been handled, you should combine all the sub-questions and all the sub-answers into a concise report, keeping the original references and reasoning in the same format and numbering. No need to make too much edition. 

Add "TERMINATE" at the end of your reply
""",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", "").replace("*", "").rstrip(), 
)

In [24]:
agents = [user_proxy, graph_rag_agent, planner, hgraph_rag_agent, engineer, critic, summarizer]
def graphRAG_speaker_selection_func(last_speaker: Agent, groupchat: autogen.GroupChat):
    """Define a customized speaker selection function.
    A recommended way is to define a transition for each speaker in the groupchat.

    Returns:
        Return an `Agent` class or a string from ['auto', 'manual', 'random', 'round_robin'] to select a default method to use.
    """
    messages = groupchat.messages
    if len(messages)==0:
        return graph_rag_agent
    else:
        if "WRITE REPORT" in messages[-1]["content"].upper():
            return summarizer

        if last_speaker is graph_rag_agent:
            return planner
        
        if last_speaker is planner:
            return hgraph_rag_agent

        elif last_speaker is graph_rag_agent:
            return engineer

        elif last_speaker is engineer:
            # for i, message in enumerate(groupchat.messages):
            #     if message['name'] == 'graph_rag_agent':
            #         groupchat.messages.pop(i)
            groupchat = [groupchat.messages[-1]]
            return critic

        elif last_speaker is critic:
            return planner
    
    return "auto"

groupchat = autogen.GroupChat(
    agents=agents,
    messages=[],
    max_round=100,
    speaker_selection_method=graphRAG_speaker_selection_func,
    enable_clear_history=True,
    # speaker_selection_method='round_robin',
)

manager = autogen.GroupChatManager(groupchat)


In [25]:
import shutil
try:
    shutil.rmtree('.cache')
except:
    pass


In [26]:
questions=[
'''
Consider a two-stage etching problem on a poly-silicon: 
In the first stage, we need to do an HAR etch with 30 nm of width and 100 nm of depth.
In the second stage, we need to perform a selectivity ratio of 22:1 (poly-silicon to SiO2) for an etch into another 45 nm of depth and the same width.  
The base is STI OX.
''',
'''
How to achieve a taper (70 degree) sidewall using Cl2/HBr/O2 gas on silicon base?
''',
'''
On a trench with a width of 15 nm and a depth of 75 nm, how to push the bottom of SiGe for 10 nm and SiP for 5 nm?
''',
'''
How to reduce iso-dense CD loading on Silicon etching?
''',
'''
How to increase the oxidizing power of O2/H2 radical on the tungsten sidewall W (width of 15 nm and depth of 75 nm) in high aspect ratio?
''',
'''
How to reduce the SiN etching at the bottom of a hole with a width of 15 nm and a depth of 70 nm without bowing?
''',
]

In [ ]:
answers = []
for q in questions:
    result = user_proxy.initiate_chat(
        manager,
        message=q,
    )
    answers.append(result)

Admin (to chat_manager):


Consider a two-stage etching problem on a poly-silicon: 
In the first stage, we need to do an HAR etch with 30 nm of width and 100 nm of depth.
In the second stage, we need to perform a selectivity ratio of 22:1 (poly-silicon to SiO2) for an etch into another 45 nm of depth and the same width.  
The base is STI OX.


--------------------------------------------------------------------------------

Next speaker: graph_rag_agent



INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Extract keywords: ['two-stage etching', 'poly-silicon', 'HAR etch', 'selectivity ratio', 'SiO2', 'STI OX']
Found [np.str_('low damage-free etching'), np.str_('polymer management'), np.str_('mram etching'), np.str_('selectivity'), np.str_('si3n4'), np.str_('o3')] in node_embeddings
Path between low damage-free etching, polymer management found as [np.str_('low damage-free etching'), 'electron-assisted chemical etching', 'semiconductor devices', 'etching process', 'ion energy distribution (ied)', 'high-frequency rf power', 'plasma density', 'narrow gap reactors', np.str_('polymer management')]
Path between low damage-free etching, mram etching found as [np.str_('low damage-free etching'), 'electron-assisted chemical etching', 'semiconductor devices', 'plasma etching', 'material properties', 'process parameters', np.str_('mram etching')]
Path between low damage-free etching, selectivity found as [np.str_('low damage-free etching'), 'electron-assisted chemical etching', 'semiconductor devi

INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"


[autogen.oai.client: 04-14 15:35:14] {351} WARNING - Model Llama3.1 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


...

[autogen.oai.client: 04-14 16:00:44] {351} WARNING - Model Llama3.1 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Engineer (to chat_manager):

QUESTION: What is the role of fluorocarbon polymers in the etching process, and how do they affect the sidewall deposition on silicon base?

ANSWER: Fluorocarbon polymers play a crucial role in the etching process by providing mask and sidewall passivation. They are deposited from the plasma and help to achieve high-quality patterns [1]. The fluorocarbon polymer gases are added to provide mask and sidewall passivation by carbon compounds, which is essential for achieving vertical mandrel profiles [2].

In addition, fluorocarbon polymers can also affect the sidewall deposition on silicon base. They can form a thin yet effective passivating layer that helps to prevent unwanted etching or damage to the silicon surface [3]. The passivation mechanisms play a crucial role in achieving high-quality patterns, and fluorocarbon polymers are an essential part of this process.

References:
[1] "Atomic Layer Processing semiconductor"
[2] "handbook-of-silicon-wafer-clean

INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"


[autogen.oai.client: 04-14 16:02:05] {351} WARNING - Model Llama3.1 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Critic (to chat_manager):

Credibility: 5/5 (The answer is based on multiple sources, including "Atomic Layer Processing semiconductor" and "handbook-of-silicon-wafer-cleaning-technology-third-edition". The information provided is directly related to the question and provides a clear understanding of the role of fluorocarbon polymers in the etching process.)

Correctness: 5/5 (The answer correctly explains the role of fluorocarbon polymers in providing mask and sidewall passivation, and how they affect the sidewall deposition on silicon base. The information provided is accurate and relevant to the question.)

Creativity: No new insights are provided in this answer.

Score: 10/10

--------------------------------------------------------------------------------

Next speaker: Planner



In [18]:
answers = []
for q in questions:
    result = user_proxy.initiate_chat(
        manager,
        message=q,
    )
    answers.append(result)

Admin (to chat_manager):


Consider a two-stage etching problem on a poly-silicon: 
In the first stage, we need to do an HAR etch with 30 nm of width and 100 nm of depth.
In the second stage, we need to perform a selectivity ratio of 22:1 (poly-silicon to SiO2) for an etch into another 45 nm of depth and the same width.  
The base is STI OX.


--------------------------------------------------------------------------------

Next speaker: graph_rag_agent



INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"


Extract keywords: ['two-stage etching', 'poly-silicon', 'HAR etch', 'selectivity ratio', 'SiO2', 'STI OX']
Found [np.str_('carrier-driven etching'), np.str_('poly-silicon films'), np.str_('mram etching'), np.str_('selectivity'), np.str_('si substrate'), np.str_('o2')] in node_embeddings
Path between carrier-driven etching, poly-silicon films found as [np.str_('carrier-driven etching'), 'etching yield', 'ion energy', 'etching process', np.str_('poly-silicon films')]
Path between carrier-driven etching, mram etching found as [np.str_('carrier-driven etching'), 'etching yield', 'reactive ion etching (rie)', 'material properties', 'process parameters', np.str_('mram etching')]
Path between carrier-driven etching, selectivity found as [np.str_('carrier-driven etching'), 'etching yield', 'ion energy', np.str_('selectivity')]
Path between carrier-driven etching, si substrate found as [np.str_('carrier-driven etching'), 'etching yield', 'ion energy', 'etching rate', 'fluorocarbon films', np.st

INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"


[autogen.oai.client: 04-14 04:15:25] {351} WARNING - Model Llama3.1 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


...

[autogen.oai.client: 04-14 06:27:36] {351} WARNING - Model Llama3.1 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Planner (to chat_manager):

I appreciate the feedback. Based on your evaluation, I will strive to provide more specific and detailed answers in the future.

Since all sub-questions have been answered, I will now conclude with a final report.

WRITE REPORT

--------------------------------------------------------------------------------

Next speaker: Summarizer



INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"


[autogen.oai.client: 04-14 06:30:54] {351} WARNING - Model Llama3.1 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Summarizer (to chat_manager):

**Final Report: Reducing SiN Etching at the Bottom of a Hole with a Width of 15 nm and a Depth of 70 nm without Bowing**

**Introduction**

The task is to reduce the SiN etching at the bottom of a hole with a width of 15 nm and a depth of 70 nm without bowing. This requires optimizing various parameters such as source power, aspect ratio, atomic layer etching (ALE), ion energy, and fluorocarbon film thickness.

**Sub-Question 1: Optimal Source Power**

The optimal source power for achieving high anisotropy while minimizing bowing can be determined by considering the relationships between source power, ion flux, and etching characteristics. A higher source power can increase ion flux, which can improve anisotropy but also increase bowing.

**Sub-Question 2: Aspect Ratio**

The high aspect ratio (AR) of 70 nm to 15 nm can significantly impact the etching process. To accommodate this high AR, adjustments can be made to the etch chemistry, environment, and pl

In [19]:
torch.device("cuda")

device(type='cuda')

In [20]:
result = user_proxy.initiate_chat(
    manager,
    message=f'''
Consider a two-stage etching problem on a poly-silicon: 
In the first stage, we need to do an HAR etch with 30 nm of width and 100 nm of depth.
In the second stage, we need to perform a selectivity ratio of 22:1 (poly-silicon to SiO2) for an etch into another 45 nm of depth and the same width.  
The base is STI OX.

We need to protect the sidewalls from etching and keep them vertical. We also need to keep sediment or debris a little as possible.
What kind of gas and combination in ratio should we apply? Any potential issue we need to consider? 

''',
)


Admin (to chat_manager):


Consider a two-stage etching problem on a poly-silicon: 
In the first stage, we need to do an HAR etch with 30 nm of width and 100 nm of depth.
In the second stage, we need to perform a selectivity ratio of 22:1 (poly-silicon to SiO2) for an etch into another 45 nm of depth and the same width.  
The base is STI OX.

We need to protect the sidewalls from etching and keep them vertical. We also need to keep sediment or debris a little as possible.
What kind of gas and combination in ratio should we apply? Any potential issue we need to consider? 



--------------------------------------------------------------------------------

Next speaker: graph_rag_agent



INFO:httpx:HTTP Request: POST http://localhost:8080/v1/chat/completions "HTTP/1.1 200 OK"


JSONDecodeError: Extra data: line 2 column 1 (char 210)

In [ ]:

agents = [user_proxy, engineer2]
def graphRAG_speaker_selection_func(last_speaker: Agent, groupchat: autogen.GroupChat):
    """Define a customized speaker selection function.
    A recommended way is to define a transition for each speaker in the groupchat.

    Returns:
        Return an `Agent` class or a string from ['auto', 'manual', 'random', 'round_robin'] to select a default method to use.
    """

    if last_speaker is planner:
        return graph_rag_agent
    
    if last_speaker is graph_rag_agent:
        return engineer
    
    if last_speaker is engineer:
        for i, message in enumerate(groupchat.messages):
            if message['name'] == 'graph_rag_agent':
                groupchat.messages.pop(i)
        return critic
    
    if last_speaker is critic:
        return planner
    
    return "auto"

groupchat = autogen.GroupChat(
    agents=agents,
    messages=[],
    max_round=2,
    speaker_selection_method='auto',
    # speaker_selection_method='round_robin',
)

manager = autogen.GroupChatManager(groupchat)


In [ ]:
print(answers)

In [ ]:
result = user_proxy.initiate_chat(
    manager,
    message=f'''
What type of gas or gas mixture is suitable for achieving a high aspect ratio (HAR) etch with 30 nm width and 100 nm depth in poly-silicon?

''',
)


In [ ]:
result = user_proxy.initiate_chat(
    manager,
    message=f'''
Consider a two-stage etching problem on a poly-silicon:
In the first stage, we need to do an HAR etch with 30 nm of width and 100 nm of depth.
In the second stage, we need to perform a selectivity ratio of 22:1 (poly-silicon to SiO2) for an etch into another 45 nm of depth and the same width.  
The base is STI OX.
We need to protect the sidewalls from etching and keep them vertical. We also need to keep sediment or debris a little as possible.
What kind of gas and combination in ratio should we apply? Any potential issue we need to consider? 


''',
)
